<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.3: Higher-Order Functions
**Prev: [Interlude: Chisel Standard Library](3.2_interlude.ipynb)**<br>
**Next: [Functional Programming](3.4_functional_programming.ipynb)**

## Motivation
前一模块中那些讨厌的 `for` 循环既冗长又违背了函数式编程的目的！在本模块中，你的生成器将变得更加函数化。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# A Tale of Two FIRs <a name="compact-fir"></a>

在上一个模块中，我们将 FIR 滤波器的卷积部分写成这样：

```scala
val muls = Wire(Vec(length, UInt(8.W)))
for(i <- 0 until length) {
  if(i == 0) muls(i) := io.in * io.consts(i)
  else       muls(i) := regs(i - 1) * io.consts(i)
}

val scan = Wire(Vec(length, UInt(8.W)))
for(i <- 0 until length) {
  if(i == 0) scan(i) := muls(i)
  else scan(i) := muls(i) + scan(i - 1)
}

io.out := scan(length - 1)
```

回顾一下，其思路是将 `io.in` 的每个元素与 `io.consts` 的相应元素相乘，并存储在 `muls` 中。
然后，将 `muls` 中的元素累加到 `scan` 中，`scan(0) = muls(0)`，`scan(1) = scan(0) + muls(1) = muls(0) + muls(1)`，一般来说 `scan(n) = scan(n-1) + muls(n) = muls(0) + ... + muls(n-1) + muls(n)`。
`scan` 中的最后一个元素（等于所有 `muls` 的总和）被赋值给 `io.out`。

然而，对于一个可能被认为相当简单的操作来说，这段代码显得非常冗长。实际上，所有这些都可以用一行代码来实现：

```scala
io.out := (taps zip io.consts).map { case (a, b) => a * b }.reduce(_ + _)
```

它在做什么呢？！让我们分解一下：
- 假设 `taps` 是所有样本的列表，其中 `taps(0) = io.in`，`taps(1) = regs(0)`，等等。
- `(taps zip io.consts)` 将两个列表 `taps` 和 `io.consts` 结合成一个列表，其中每个元素是对应位置上的元素的元组。具体来说，它的值将是 `[(taps(0), io.consts(0)), (taps(1), io.consts(1)), ..., (taps(n), io.consts(n))]`。请记住，句点是可选的，所以这相当于 `(taps.zip(io.consts))`。
- `.map { case (a, b) => a * b }` 将匿名函数（接受一个包含两个元素的元组并返回它们的乘积）应用于列表的每个元素，并返回结果。在这个例子中，结果相当于冗长示例中的 `muls`，其值为 `[taps(0) * io.consts(0), taps(1) * io.consts(1), ..., taps(n) * io.consts(n)]`。你将在下一个模块中重新学习匿名函数。现在，只需了解这种语法。
- 最后，`.reduce(_ + _)` 也将函数（元素相加）应用于列表的元素。然而，它接受两个参数：第一个是当前的累加值，第二个是列表元素（在第一次迭代中，它只是将前两个元素相加）。这些参数由括号中的两个下划线表示。然后假设从左到右遍历，结果将是 `(((muls(0) + muls(1)) + muls(2)) + ...) + muls(n)`，其中嵌套较深的括号首先求值。这就是卷积的输出。

---
# Functions as Arguments

正式来说，像 `map` 和 `reduce` 这样的函数被称为 _higher-order functions_：它们是以函数作为参数的函数。
事实证明（希望你能从上面的例子中看到），这些是非常强大的结构，它们封装了一般的计算模式，允许你专注于应用逻辑而不是流控制，从而产生非常简洁的代码。

## Different ways of specifying functions

你可能已经注意到，上面的例子中有两种指定函数的方法：
- 对于每个参数只引用一次的函数，你*可以*使用下划线（`_`）引用每个参数。在上面的例子中，`reduce` 参数函数接受两个参数，可以指定为 `_ + _`。虽然很方便，但这需要遵循一组额外的复杂规则，所以如果它不起作用，请尝试：
- 显式指定输入参数列表。reduce 可以显式写成 `(a, b) => a + b`，一般形式是将参数列表放在括号中，后跟 `=>`，然后是引用这些参数的函数体。
- 当需要元组解包时，使用 `case` 语句，例如 `case (a, b) => a * b`。它接受一个参数，一个包含两个元素的元组，并将其解包到变量 `a` 和 `b` 中，然后可以在函数体中使用这些变量。

## Practice in Scala

在上一个模块中，我们已经见过 Scala Collections API 中的主要类，如 `List`。
这些高阶函数是这些 API 的一部分——实际上，上面的例子使用了 `List` 的 `map` 和 `reduce` API。
在本节中，我们将通过例子和练习熟悉这些方法。
在这些例子中，为了简单和清晰起见，我们将操作 Scala 数字（`Int`），但由于 Chisel 操作符的行为类似，这些概念应该可以推广。

<span style="color:blue">**示例：Map**</span><br>
`List[A].map` 的类型签名是 `map[B](f: (A) ⇒ B): List[B]`。你将在后面的模块中学习更多关于类型的知识。现在，可以将类型 A 和 B 理解为 `Int` 或 `UInt`，这意味着它们可以是软件或硬件类型。

用简单的语言来说，它接受一个类型为 `(f: (A) ⇒ B)` 的参数，或者一个接受类型为 `A` 的参数（与输入 List 的元素类型相同）并返回类型为 `B` 的值（可以是任何东西）的函数。然后 `map` 返回一个新的类型为 `B` 的列表（参数函数的返回类型）。

由于我们已经解释了 FIR 示例中 List 的行为，让我们直接进入示例和练习：

In [ ]:
println(List(1, 2, 3, 4).map(x => x + 1))  // explicit argument list in function
println(List(1, 2, 3, 4).map(_ + 1))  // equivalent to the above, but implicit arguments
println(List(1, 2, 3, 4).map(_.toString + "a"))  // the output element type can be different from the input element type

println(List((1, 5), (2, 6), (3, 7), (4, 8)).map { case (x, y) => x*y })  // this unpacks a tuple, note use of curly braces

// Related: Scala has a syntax for constructing lists of sequential numbers
println(0 to 10)  // to is inclusive , the end point is part of the result
println(0 until 10)  // until is exclusive at the end, the end point is not part of the result

// Those largely behave like lists, and can be useful for generating indices:
val myList = List("a", "b", "c", "d")
println((0 until 4).map(myList(_)))

<span style="color:red">**Exercise: Map**</span><br><a name="map-exercise"></a>

In [ ]:
// Now you try: 
// Fill in the blanks (the ???) such that this doubles the elements of the input list.
// This should return: List(2, 4, 6, 8)
println(List(1, 2, 3, 4).map(???))

<span style="color:blue">**示例：zipWithIndex**</span><br>
`List.zipWithIndex` 的类型签名是 `zipWithIndex: List[(A, Int)]`。

它不接受任何参数，但返回一个列表，其中每个元素是原始元素和索引的元组（第一个索引为零）。
所以 `List("a", "b", "c", "d").zipWithIndex` 将返回 `List(("a", 0), ("b", 1), ("c", 2), ("d", 3))`

当在某些操作中需要元素索引时，这非常有用。

由于这非常简单明了，我们就举几个例子：

In [ ]:
println(List(1, 2, 3, 4).zipWithIndex)  // note indices start at zero
println(List("a", "b", "c", "d").zipWithIndex)
println(List(("a", "b"), ("c", "d"), ("e", "f"), ("g", "h")).zipWithIndex)  // tuples nest

<span style="color:blue">**示例：Reduce**</span><br>
`List[A].map` 的类型签名类似于 `reduce(op: (A, A) ⇒ A): A`。（实际上更宽松，`A` 只需要是列表类型的超类型，但我们在这里不处理该语法）

由于上面也解释了它的工作原理，这里有一些例子：

In [ ]:
println(List(1, 2, 3, 4).reduce((a, b) => a + b))  // returns the sum of all the elements
println(List(1, 2, 3, 4).reduce(_ * _))  // returns the product of all the elements
println(List(1, 2, 3, 4).map(_ + 1).reduce(_ + _))  // you can chain reduce onto the result of a map

In [ ]:
// Important note: reduce will fail with an empty list
println(List[Int]().reduce(_ * _))

<span style="color:red">**Exercise: Reduce**</span><br><a name="reduce-exercise"></a>

In [ ]:
// Now you try: 
// Fill in the blanks (the ???) such that this returns the product of the double of the elements of the input list.
// This should return: (1*2)*(2*2)*(3*2)*(4*2) = 384
println(List(1, 2, 3, 4).map(???).reduce(???))

<span style="color:blue">**示例：Fold**</span><br>
`List[A].fold` 与 reduce 非常相似，只是你可以指定初始累加值。
它的类型签名类似于 `fold(z: A)(op: (A, A) ⇒ A): A`。（像 `reduce` 一样，`A` 的类型也更加宽松）

值得注意的是，它接受两个参数列表，第一个 (`z`) 是初始值，第二个是累加函数。
与 `reduce` 不同，它在列表为空时不会失败，而是直接返回初始值。

这里有一些例子：

In [ ]:
println(List(1, 2, 3, 4).fold(0)(_ + _))  // equivalent to the sum using reduce
println(List(1, 2, 3, 4).fold(1)(_ + _))  // like above, but accumulation starts at 1
println(List().fold(1)(_ + _))  // unlike reduce, does not fail on an empty input

<span style="color:red">**Exercise: Fold**</span><br><a name="fold-exercise"></a>

In [ ]:
// Now you try: 
// Fill in the blanks (the ???) such that this returns the double the product of the elements of the input list.
// This should return: 2*(1*2*3*4) = 48
// Note: unless empty list tolerance is needed, reduce is a much better fit here.
println(List(1, 2, 3, 4).fold(???)(???))

<span style="color:red">**练习：Decoupled Arbiter**</span><br>
现在让我们把所有东西整合到一个练习中。

在这个例子中，我们将构建一个 Decoupled 仲裁器：一个具有 _n_ 个 Decoupled 输入和一个 Decoupled 输出的模块。
仲裁器选择最低通道的有效信号并将其转发到输出。

一些提示：
- 架构上：
  - 如果任何输入有效，`io.out.valid` 为真
  - 考虑有一个内部选择通道的线
  - 每个输入的 `ready` 在输出准备好且该通道被选择时为真（这会组合连接 ready 和 valid，但我们现在先忽略它……）
- 这些结构可能会有所帮助：
  - `map`，特别是用于返回子元素的 Vec，例如 `io.in.map(_.valid)` 返回输入 Bundles 的有效信号列表
  - `PriorityMux(List[Bool, Bits])`，它接受有效信号和位的列表，返回第一个有效的元素
  - 使用 UInt 对 Vec 进行动态索引，例如 `io.in(0.U)`

In [ ]:
class MyRoutingArbiter(numChannels: Int) extends Module {
  val io = IO(new Bundle {
    val in = Vec(numChannels, Flipped(Decoupled(UInt(8.W))))
    val out = Decoupled(UInt(8.W))
  } )

  // YOUR CODE BELOW
  ???
}

test(new MyRoutingArbiter(4)) { c =>
    // verify that the computation is correct
    // Set input defaults
    for(i <- 0 until 4) {
        c.io.in(i).valid.poke(false.B)
        c.io.in(i).bits.poke(i.U)
        c.io.out.ready.poke(true.B)
    }

    c.io.out.valid.expect(false.B)

    // Check single input valid behavior with backpressure
    for (i <- 0 until 4) {
        c.io.in(i).valid.poke(true.B)
        c.io.out.valid.expect(true.B)
        c.io.out.bits.expect(i.U)

        c.io.out.ready.poke(false.B)
        c.io.in(i).ready.expect(false.B)

        c.io.out.ready.poke(true.B)
        c.io.in(i).valid.poke(false.B)
    }

    // Basic check of multiple input ready behavior with backpressure
    c.io.in(1).valid.poke(true.B)
    c.io.in(2).valid.poke(true.B)
    c.io.out.bits.expect(1.U)
    c.io.in(1).ready.expect(true.B)
    c.io.in(0).ready.expect(false.B)

    c.io.out.ready.poke(false.B)
    c.io.in(1).ready.expect(false.B)
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
class MyRoutingArbiter(numChannels: Int) extends Module {
  val io = IO(new Bundle {
    val in = Vec(numChannels, Flipped(Decoupled(UInt(8.W))))
    val out = Decoupled(UInt(8.W))
  } )

  // YOUR CODE BELOW
  io.out.valid := io.in.map(\_.valid).reduce(\_ || \_)
  val channel = PriorityMux(
    io.in.map(\_.valid).zipWithIndex.map { case (valid, index) => (valid, index.U) }
  )
  io.out.bits := io.in(channel).bits
  io.in.map(\_.ready).zipWithIndex.foreach { case (ready, index) =>
    ready := io.out.ready && channel === index.U
  }
}
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)